## [Meta Coding Puzzles](https://www.metacareers.com/profile/coding_puzzles/)

Below are my solutions for the "Level 2" puzzles.

### Director of Photography (Chapter 2)
Time limit: 5s

*Note: Chapter 1 is an easier version of this puzzle. The only difference is a smaller constraint on $N$.*

A photography set consists of $N$ cells in a row, numbered from $1$ to $N$ in order, and can be represented by a string $C$ of length $N$. Each cell $i$ is one of the following types (indicated by $C_i$, the $i$th character of $C$):

* If $C_i$ = “P”, it is allowed to contain a photographer
* If $C_i$ = “A”, it is allowed to contain an actor
* If $C_i$ = “B”, it is allowed to contain a backdrop
* If $C_i$ = “.”, it must be left empty

A photograph consists of a photographer, an actor, and a backdrop, such that each of them is placed in a valid cell, and such that the actor is between the photographer and the backdrop. Such a photograph is considered artistic if the distance between the photographer and the actor is between $X$ and $Y$ cells (inclusive), and the distance between the actor and the backdrop is also between $X$ and $Y$ cells (inclusive). The distance between cells $i$ and $j$ is $∣i−j∣$ (the absolute value of the difference between their indices).

Determine the number of different artistic photographs which could potentially be taken at the set. Two photographs are considered different if they involve a different photographer cell, actor cell, and/or backdrop cell.

**Constraints**

$1 \leq N \leq 300{,}000 \\
1 \leq X \leq Y \leq N$

**Sample test cases**
1. $N = 5 \\
C = APABA \\
X = 1 \\
Y = 2$

&emsp;&emsp;Expected return value = 1

2. $N = 5 \\
C = APABA \\
X = 2 \\
Y = 3$

&emsp;&emsp;Expected return value = 0

3. $N = 8 \\
C = \, .PBAAP.B \\
X = 1 \\
Y = 3$

&emsp;&emsp;Expected return value = 3

#### Solution

Now, the upper bound on $N$ is $300{,}000$ rather than $200$. My last solution looped through $N^3$ combinations of $P,A,B$ locations, so with a large $N$ this may be too slow given the time limit. Rather than try every combination, I should be able to speed things up by looking only at values I know will be acceptable positions of each $P, A, B$. Rather than checking all $N$ entries for each, I can use `numpy.where()` to extract indices that are potential placements of each element. Since each element $C_i$ can only be one value, each index can only appear for at most one of $P, A, B$. 

I can also reduce the loops by using the artistic photograph conditions. For $P$ at $C_P$, $A$ at $C_A$ has to be within $[C_P - Y, C_P - X]$ or $[C_P + X, C_P + Y]$, i.e. $|C_P - C_A| \geq X$ and $|C_P - C_A| \leq Y$. Similarly, for $B$ at $C_B$, $|C_B - C_A| \geq X$ and $|C_B - C_A| \leq Y$. So, for a given $C_A$, I only have to consider a subset of possible $C_P, C_B$. Also, if $C_P > C_A$, then $C_B < C_A$ is required and vice versa.

This will greatly reduce the number of times through all the loops required.

In [1]:
import numpy as np

In [43]:
# helper function for checking whether a given orientation is a(n artistic) photograph
# to do this, I just need to index of each element and the X and Y values
# first I'll check for valid photograph, returning false if it is not valid
# then I'll check for artistic photograph, returning false if it is not artistic
# finally, if it passes all requirements, return true
def checkArtisticPhotograph(P: int, A: int, B: int, X: int, Y: int) -> bool:
    # check orientation, A needs to be in between P and B
    if P < B and (A > B or A < P):
        return False
    elif B < P and (A < B or A > P):
        return False
    else:
        # orientation valid, check for distances
        if abs(A - P) < X or abs(A - P) > Y:
            return False
        elif abs(A - B) < X or abs(A - B) > Y:
            return False
    # passed all checks
    return True

def getArtisticPhotographCount(N: int, C: str, X: int, Y: int) -> int:
    tStart = time.time()
    # list comprehension gets bool array of each allowed index for P, A, B
    pBool = [True if x=='P' else False for x in C]
    aBool = [True if x=='A' else False for x in C]
    bBool = [True if x=='B' else False for x in C]
    # numpy.where() returns indices for allowed P, A, B
    pVals = np.where(pBool)[0]
    aVals = np.where(aBool)[0]
    bVals = np.where(bBool)[0]
    
    # counter for artistic photographs
    nArtisticPhotos = 0
    
    # now loop through all positions for A, P, B separately
    # limit search to artistic locations of P, B given A
    # skip combinations that don't meet distance and orientation requirements
    # increment counter if all satisfied
    for a in aVals:
        print('Looping for A position '+ str(a) + '. Total time elapsed = ' + str(time.time() - tStart))
        for p in pVals:
            if abs(p - a) < X or abs(p - a) > Y:
                continue
            for b in bVals:
                if abs(b - a) < X or abs(b - a) > Y:
                    continue
                # skip if P, B on same side of A
                if (p > a and b > a) or (p < a and b < a):
                    continue
                nArtisticPhotos += 1
    
    return nArtisticPhotos

In [40]:
print(getArtisticPhotographCount(5, 'APABA', 1, 2))
print(getArtisticPhotographCount(5, 'APABA', 2, 3))
print(getArtisticPhotographCount(8, '.PBAAP.B', 1, 3))

1
0
3


The simple test cases are solved, but I need to check complex cases where $N$ is at its upper bound. I'll check how long it takes to handle such a case.

In [41]:
import time

In [42]:
import random

In [44]:
vals = ['P','A','B','.']
N = 300000
C = random.choices(vals, k=N)
tStart = time.time()
print(getArtisticPhotographCount(N, C, 500, 1000))
tTot = time.time() - tStart
print(tTot)

Looping for A position 3. Total time elapsed = 0.09955120086669922
Looping for A position 6. Total time elapsed = 6.243028163909912
Looping for A position 7. Total time elapsed = 11.660627126693726
Looping for A position 11. Total time elapsed = 18.400877237319946


KeyboardInterrupt: 

There's still an issue. Even trying to limit the operations, the issue is still with actually looping through everything. It would be nicer possibly if I could change the `for` loop with the given conditions that are checked, rather than checking all ~100,000 values against the conditions.

Some more simplifications I can make:
* $C_A$ has to be at least $X$ cells from the endpoints

In [47]:
def getArtisticPhotographCount(N: int, C: str, X: int, Y: int) -> int:
    tStart = time.time()
    # list comprehension gets bool array of each allowed index for P, A, B
    pBool = [True if x=='P' else False for x in C]
    aBool = [True if x=='A' else False for x in C]
    bBool = [True if x=='B' else False for x in C]
    # numpy.where() returns indices for allowed P, A, B
    pVals = np.where(pBool)[0]
    aVals = np.where(aBool)[0]
    bVals = np.where(bBool)[0]
    # A needs at least X distance to left/right
    aVals = aVals[np.where((aVals >= X) & (aVals <= N-X))[0]]
    #print(pVals, aVals, bVals)
    
    # counter for artistic photographs
    nArtisticPhotos = 0
    
    # now loop through all positions for A
    # loop through acceptable P, B positions given A
    # and check that they are in the allowed positions for P, B
    # increment counter if all satisfied
    for a in aVals:
        print('Looping for A position '+ str(a) + '. Total time elapsed = ' + str(time.time() - tStart))
        # P to left of A
        for i in range(a-Y, a-X+1):
            if i in pVals:
                # B must be to right of A
                for j in range(a+X, a+Y+1):
                    if j in bVals:
                        nArtisticPhotos += 1
        # P to right of A
        for i in range(a+X, a+Y+1):
            if i in pVals:
                # B must be to left of A
                for j in range(a-Y, a-X+1):
                    if j in bVals:
                        nArtisticPhotos += 1
    
    return nArtisticPhotos

In [46]:
print(getArtisticPhotographCount(5, 'APABA', 1, 2))
print(getArtisticPhotographCount(5, 'APABA', 2, 3))
print(getArtisticPhotographCount(8, '.PBAAP.B', 1, 3))

1
0
3


In [48]:
vals = ['P','A','B','.']
N = 300000
C = random.choices(vals, k=N)
tStart = time.time()
print(getArtisticPhotographCount(N, C, 500, 1000))
tTot = time.time() - tStart
print(tTot)

Looping for A position 503. Total time elapsed = 0.11040210723876953
Looping for A position 513. Total time elapsed = 2.636223077774048
Looping for A position 514. Total time elapsed = 5.095998287200928
Looping for A position 518. Total time elapsed = 7.399182081222534


KeyboardInterrupt: 

It's improving (cut down positions to check for $A$, time to check went from ~6-7s to 2-3s), but still too slow.

One improvement could be to store $P$, $B$ as sets for faster checks/lookups.

But, I don't actually have to do a loop over positions. Checking one by one is unnecessary if I know that the placements work. $P$ and $B$ are independent of each other. As long as $P$, $B$ individually meet the distance requirements with $A$, it doesn't matter how they relate. Therefore, I don't need to loop over every $B$ for every $P$. I can just count the number of good placements for $B$ and the number of good placements for $P$ for a given $A$ and multiply.

To do this, I will store cumulative sums of $P$, $B$ locations so that for a given a position, I can easily calculate the number of $P$, $B$ options in the artistic range without any looping. This enable me to go from nested loops for every $A, P, B$ combination to just looping each once, then retrieving elements of the cumulative sum arrays.

The cumulative sum at position $k$ is the sum of all entries up to and including $k$. So to get the sum in array $L$ from positions $k_1$ to $k_2$, I'll want $L[k_2] - L[k_1 - 1]$. Since the $X$ to $Y$ range I'm interested in for this problem may extend beyond the limits of the arrays, I'll have to be careful to handle these cases.

In [49]:
def getArtisticPhotographCount(N: int, C: str, X: int, Y: int) -> int:
    # list comprehension gets bool array of each allowed index for P, A, B
    pBool = [True if x=='P' else False for x in C]
    aBool = [True if x=='A' else False for x in C]
    bBool = [True if x=='B' else False for x in C]
    # numpy.where() returns indices for allowed A
    aVals = np.where(aBool)[0]
    # A needs at least X distance to left/right
    # indices go 0 to N-1, so A should go from 0+X to N-1-X
    aVals = aVals[np.where((aVals >= X) & (aVals <= N-1-X))[0]]

    # numpy.cumsum() returns sum of allowed P,B positions up to index
    pSums = np.cumsum(pBool)
    bSums = np.cumsum(bBool)

    # counter for artistic photographs
    nArtisticPhotos = 0
    
    # now loop through all positions for A
    # for each, calculate # allowed P, B in artistic range
    for a in aVals:
        # P,B to left of A
        # if a-Y extends to beginning of array, then a-X value just gives sum up to there
        if (a-Y-1) <= 0:
            leftP = pSums[a-X]
            leftB = bSums[a-X]
        else:
            leftP = pSums[a-X] - pSums[a-Y-1]
            leftB = bSums[a-X] - bSums[a-Y-1]
        #P,B to right of A
        # if a+Y extends to end of array, sum is just from a+X-1 to end of array
        if (a+Y) >= N-1:
            rightP = pSums[N-1] - pSums[a+X-1]
            rightB = bSums[N-1] - bSums[a+X-1]
        else:
            rightP = pSums[a+Y] - pSums[a+X-1]
            rightB = bSums[a+Y] - bSums[a+X-1]
        # combine left (right) P with right (left) B
        nArtisticPhotos += leftP*rightB
        nArtisticPhotos += rightP*leftB
    
    return nArtisticPhotos

In [50]:
print(getArtisticPhotographCount(5, 'APABA', 1, 2))
print(getArtisticPhotographCount(5, 'APABA', 2, 3))
print(getArtisticPhotographCount(8, '.PBAAP.B', 1, 3))

1
0
3


In [51]:
vals = ['P','A','B','.']
N = 300000
C = random.choices(vals, k=N)
tStart = time.time()
print(getArtisticPhotographCount(N, C, 500, 1000))
tTot = time.time() - tStart
print(tTot)

2340184114
0.45047616958618164


It works quickly for this complicated case! Much, much faster to not have to use nested loops. I got caught in the nested loop mindset because it seemed natural for checking combinations of $P, A, B$, but this could be solved much more simply without having to do that.

**Results: 34/39 test cases solved**

Hm, my output was wrong on 5/39 cases. I'll have to come back to this later to see if I can figure out what's going wrong.

*To be continued*

### Hops
Time limit: 5s

A family of frogs in a pond are traveling towards dry land to hibernate. They hope to do so by hopping across a trail of $N$ lily pads, numbered from $1$ to $N$ in order.

There are $F$ frogs, numbered from $1$ to $F$. Frog $i$ is currently perched atop lily pad $P_i$. No two frogs are currently on the same lily pad. Lily pad $N$ is right next to the shore, and none of the frogs are initially on lily pad $N$.

Each second, one frog may hop along the trail towards lily pad $N$. When a frog hops, it moves to the nearest lily pad after its current lily pad which is not currently occupied by another frog (hopping over any other frogs on intermediate lily pads along the way). If this causes it to reach lily pad $N$, it will immediately exit onto the shore. Multiple frogs may not simultaneously hop during the same second.

Assuming the frogs work together optimally when deciding which frog should hop during each second, determine the minimum number of seconds required for all $F$ of them to reach the shore.

**Constraints**

$2 \leq N \leq 10^{12} \\
1 \leq F \leq 500{,}000 \\
1 \leq P_i \leq N - 1$

**Sample test cases**
1. $N = 3 \\
F = 1 \\
P = [1]$

&emsp;&emsp;Expected return value = 2

2. $N = 6 \\
F = 3 \\
P = [5, 2, 4]$

&emsp;&emsp;Expected return value = 4

#### Solution

### Missing Mail
Time limit: 15s

You are the manager of a mail room which is frequently subject to theft. A period of $N$ days is about to occur, such that on the $i$th day, the following sequence of events will occur in order:
* A package with a value of $V_i$ dollars will get delivered to the mail room (unless $V_i = 0$, in which case no package will get delivered).
* You can choose to pay $C$ dollars to enter the mail room and collect all of the packages there (removing them from the room), and then leave the room
* With probability $S$, all packages currently in the mail room will get stolen (and therefore removed from the room).

Note that you're aware of the delivery schedule $V_{1..N}$, but can only observe the state of the mail room when you choose to enter it, meaning that you won't immediately be aware of whether or not packages were stolen at the end of any given day.

Your profit after the $N$th day will be equal to the total value of all packages which you collected up to that point, minus the total amount of money you spent on entering the mail room.

Please determine the maximum expected profit you can achieve (in dollars).

Note: Your return value must have an absolute or relative error of at most $10^{-6}$ to be considered correct.

**Constraints**
$1 \leq N \leq 4{,}000 \\
0 \leq V_i \leq 1{,}000 \\
1 \leq C \leq 1{,}000 \\
0.0 \leq S \leq 1.0$

**Sample test cases**
1. $N = 5 \\
V = [10, 2, 8, 6, 4] \\
C = 5 \\
S = 0.0$

&emsp;&emsp;Expected return value = 25.00000000

2. $N = 5 \\
V = [10, 2, 8, 6, 4] \\
C = 5 \\
S = 1.0$

&emsp;&emsp;Expected return value = 9.00000000

3. $N = 5 \\
V = [10, 2, 8, 6, 4] \\
C = 3 \\
S = 0.5$

&emsp;&emsp;Expected return value = 17.00000000

4. $N = 5 \\
V = [10, 2, 8, 6, 4] \\
C = 3 \\
S = 0.15$

&emsp;&emsp;Expected return value = 20.10825000

#### Solution

### Portals
Time limit: 5s

You've found yourself in a grid of cells with $R$ rows and $C$ columns. The cell in the $i$th row from the top and $j$th column from the left contains one of the following (indicated by the character $G_{i,j}$):
* If $G_{i,j}$ = ".", the cell is empty.
* If $G_{i,j}$ = "S", the cell contains your starting position. There is exactly one such cell.
* If $G_{i,j}$ = "E", the cell contains an exit. There is at least one such cell.
* If $G_{i,j}$ = "#", the cell contains a wall.
* Otherwise, if $G_{i,j}$ is a lowercase letter (between "a" and "z", inclusive), the cell contains a portal marked with that letter.

Your objective is to reach any exit from your starting position as quickly as possible. Each second, you may take either of the following actions:
* Walk to a cell adjacent to your current one (directly above, below, to the left, or to the right), as long as you remain within the grid and that cell does not contain a wall.
* If your current cell contains a portal, teleport to any other cell in the grid containing a portal marked with the same letter as your current cell's portal.

Determine the minimum number of seconds required to reach any exit, if it's possible to do so at all. If it's not possible, return −1 instead.

**Constraints**

$1 \leq R, C \leq 50 \\$
$G_{i,j} \in$ {".","S","E","#","a"$\dots$"z"}

**Sample test cases**
1. $R = 3 \\
C = 3 \\
G = \begin{matrix}
    \text{.E.} \\
    \text{.#E} \\
    \text{.S#}
    \end{matrix}$

&emsp;&emsp;Expected return value = 4

2. $R = 3 \\
C = 4 \\
G = \begin{matrix}
    \text{a.Sa} \\
    \text{####} \\
    \text{Eb.b}
    \end{matrix}$

&emsp;&emsp;Expected return value = -1

3. $R = 3 \\
C = 4 \\
G = \begin{matrix}
    \text{aS.b} \\
    \text{####} \\
    \text{Eb.a}
    \end{matrix}$

&emsp;&emsp;Expected return value = 4

4. $R = 1 \\
C = 9 \\
G = \begin{matrix}
\text{xS..x..Ex}
\end{matrix}$

&emsp;&emsp;Expected return value = 3

#### Solution

### Rabbit Hole (Chapter 1)
Time limit: 5s

*Note: Chapter 2 is a harder version of this puzzle.*

You're having a grand old time clicking through the rabbit hole that is your favorite online encyclopedia.

The encyclopedia consists of $N$ different web pages, numbered from 
$1$ to $N$. Each page $i$ contains nothing but a single link to a different page $L_i$.

A session spent on this website involves beginning on one of the $N$ pages, and then navigating around using the links until you decide to stop. That is, while on page $i$, you may either move to page $L_i$, or stop your browsing session.

Assuming you can choose which page you begin the session on, what's the maximum number of different pages you can visit in a single session? Note that a page only counts once even if visited multiple times during the session.

**Constraints**
$2 \leq N \leq 500{,}000 \\
1 \leq L_i \leq N \\
L_i \neq i$

**Sample test cases**
1. $N = 4 \\
L = [4, 1, 2, 1]$

&emsp;&emsp;Expected return value = 4

2. $N = 5 \\
L = [4, 3, 5, 1, 2]$

&emsp;&emsp;Expected return value = 3

3. $N = 5 \\
L = [2, 4, 2, 2, 3]$

&emsp;&emsp;Expected return value = 4

#### Solution

*try: for each case, store loops in sets, if you can get to one page in the set you can eventually get to all other pages in that set. go through all pages then return size of largest set*

### Rotary Lock (Chapter 2)
Time limit: 15s

*Note: Chapter 1 is an easier version of this puzzle.*

You're trying to open a lock. The lock comes with two wheels, each of which has the integers from $1$ to $N$ arranged in a circle in order around it (with integers $1$ and $N$ adjacent to one another). Each wheel is initially pointing at $1$.

It takes $1$ second to rotate a wheel by $1$ unit to an adjacent integer in either direction, and it takes no time to select an integer once a wheel is pointing at it.

The lock will open if you enter a certain code. The code consists of a sequence of $M$ integers, the $i$th of which is $C_i$. For each integer in the sequence, you may select it with either of the two wheels. Determine the minimum number of seconds required to select all $M$ of the code's integers in order.

**Constraints**
$3 \leq N \leq 1{,}000{,}000{,}000 \\
1 \leq M \leq 3{,}000 \\
1 \leq C_i \leq N$

**Sample test cases**
1. $N = 3 \\
M = 3 \\
C = [1, 2, 3]$

&emsp;&emsp;Expected return value = 2

2. $N = 10 \\
M = 4 \\
C = [9, 4, 4, 8]$

&emsp;&emsp;Expected return value = 6

#### Solution

### Scoreboard Inference (Chapter 2)
Time limit: 5s

*Note: Chapter 1 is an easier version of this puzzle. The only difference is a smaller set of possible problem point values.*

You are spectating a programming contest with $N$ competitors, each trying to independently solve the same set of programming problems. Each problem has a point value, which is **either 1, 2, or 3**.

On the scoreboard, you observe that the $i$th competitor has attained a score of $S_i$, which is a positive integer equal to the sum of the point values of all the problems they have solved.

The scoreboard does not display the number of problems in the contest, nor their point values. Using the information available, you would like to determine the minimum possible number of problems in the contest.

**Constraints**
$1 \leq N \leq 500{,}000 \\
1 \leq S_i \leq 1{,}000{,}000{,}000$

**Sample test cases**
1. $N = 5 \\
S = [1, 2, 3, 4, 5]$

&esmp;&emsp;Expected return value = 3

2. $N = 4 \\
S = [4, 3, 3, 4]$

&emsp;&emsp;Expected return value = 2

3. $N = 4 \\
S = [2, 4, 6, 8]$

&emsp;&emsp;Expected return value = 4

4. $N = 1 \\
S = [8]$

&emsp;&emsp;Expected return value = 3

#### Solution

### Tunnel Time
Time limit: 5s

There’s a circular train track with a circumference of $C$ metres. Positions along the track are measured in metres, clockwise from its topmost point. For example, the topmost point is position $0$, $1$ metre clockwise along the track is position $1$, and $1$ metre counterclockwise along the track is position $C−1$.

A train with negligible length runs clockwise along this track at a speed of $1$ metre per second, starting from position $0$. After $C$ seconds go by, the train will have driven around the entire track and returned to position $0$, at which point it will continue going around again, with this process repeated indefinitely.

There are $N$ tunnels covering sections of the track, the $i$th of which begins at position $A_i$ and ends at position $B_i$ (and therefore has a length of $B_i - A_i$ metres). No tunnel covers or touches position $0$ (including at its endpoints), and no two tunnels intersect or touch one another (including at their endpoints). For example, if there's a tunnel spanning the interval of positions $[1,4]$, there cannot be another tunnel spanning intervals $[2,3]$ nor $[4,5]$.

The train’s *tunnel time* is the total number of seconds it has spent going through tunnels so far. Determine the total number of seconds which will go by before the train’s tunnel time becomes $K$.

**Constraints**
$3 \leq C \leq 10^{12} \\
1 \leq N \leq 500{,}000 \\
1 \leq A_i < B_i \leq C - 1 \\
1 \leq K \leq 10^{12}$

Each test case's parameters will be formulated such that the correct answer for that case is at most $10^{15}$.

**Sample test cases**
1. $C = 10 \\
N = 2 \\
A = [1, 6] \\
B = [3, 7] \\
K = 7$

&emsp;&emsp;Expected return value = 22

2. $C = 50 \\
N = 3 \\
A = [39, 19, 28] \\
B = [49, 27, 35] \\
K = 15$

&emsp;&emsp;Expected return value = 35

#### Solution